# TFA: Time-Frequency Analysis (WIP)

A work in progress for the new interface for TFA, just showing the data access and process application.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np

from swarmpal.io import create_paldata, PalDataItem
from swarmpal.toolboxes import tfa

## MAGx_LR example

### Fetching data

In [ ]:
data_params = dict(
    collection="SW_OPER_MAGA_LR_1B",
    measurements=["B_NEC", "F"],
    models=["Model='CHAOS-Core'+'CHAOS-Static'"],
    start_time=dt.datetime(2015, 3, 14),
    end_time=dt.datetime(2015, 3, 14, 3, 59, 59),
    pad_times=(dt.timedelta(hours=3), dt.timedelta(hours=3)),
    server_url="https://vires.services/ows",
    options=dict(asynchronous=False, show_progress=False),
)
data = create_paldata(PalDataItem.from_vires(**data_params))
print(data)

### Applying processes

#### TFA: Preprocess

In [ ]:
p1 = tfa.processes.Preprocess()
p1.set_config(
    dataset="SW_OPER_MAGA_LR_1B",
    active_variable="B_MFA",
    active_component=2,
    sampling_rate=1,
    remove_model=True,
    convert_to_mfa=True,
)
p1(data)
print(data)

In [ ]:
fig, axes = plt.subplots(3, 1, sharex=True)
data["SW_OPER_MAGA_LR_1B"]["B_NEC"].plot.line(x="Timestamp", ax=axes[0])
data["SW_OPER_MAGA_LR_1B"]["B_NEC_res_Model"].plot.line(x="Timestamp", ax=axes[1])
data["SW_OPER_MAGA_LR_1B"]["B_MFA"].plot.line(x="Timestamp", ax=axes[2])
axes[1].set_ylim(-200, 200)
axes[2].set_ylim(-200, 200);

In [ ]:
data["SW_OPER_MAGA_LR_1B"]["TFA_Variable"].plot.line(x="TFA_Time");

#### TFA: Clean

In [ ]:
p2 = tfa.processes.Clean()
p2.set_config(
    window_size=300,
    method="iqr",
    multiplier=1,
)
p2(data)
data["SW_OPER_MAGA_LR_1B"]["TFA_Variable"].plot.line(x="TFA_Time");

#### TFA: Filter

In [ ]:
p3 = tfa.processes.Filter()
p3.set_config(
    cutoff_frequency=20 / 1000,
)
p3(data)
data["SW_OPER_MAGA_LR_1B"]["TFA_Variable"].plot.line(x="TFA_Time");

#### TFA: Wavelet

In [ ]:
p4 = tfa.processes.Wavelet()
p4.set_config(
    min_frequency=20 / 1000,
    max_frequency=100 / 1000,
    dj=0.1,
)
p4(data)
print(data)

### Plotting

Note that by default (see `clip_times=True`), these plots are restricted to the analysis window (the full time window minus the time pads added in the data request).

In [ ]:
tfa.plotting.time_series(data)

In [ ]:
tfa.plotting.spectrum(data, levels=np.linspace(-6, -1, 20))

In [ ]:
tfa.plotting.quicklook(data)

## MAGx_HR

In [ ]:
data_params = dict(
    collection="SW_OPER_MAGB_HR_1B",
    measurements=["B_NEC"],
    models=["Model='CHAOS-Core'+'CHAOS-Static'"],
    start_time=dt.datetime(2015, 3, 14, 12, 5, 0),
    end_time=dt.datetime(2015, 3, 14, 12, 30, 0),
    pad_times=(dt.timedelta(minutes=10), dt.timedelta(minutes=10)),
    server_url="https://vires.services/ows",
    options=dict(asynchronous=False, show_progress=False),
)
data = create_paldata(PalDataItem.from_vires(**data_params))

In [ ]:
p1 = tfa.processes.Preprocess()
p1.set_config(
    dataset="SW_OPER_MAGB_HR_1B",
    active_variable="B_NEC_res_Model",
    sampling_rate=50,
    remove_model=True,
    use_magnitude=True,
)
p2 = tfa.processes.Clean()
p2.set_config(
    window_size=300,
    method="iqr",
    multiplier=1,
)
p3 = tfa.processes.Filter()
p3.set_config(
    cutoff_frequency=0.1,
)
p4 = tfa.processes.Wavelet()
p4.set_config(
    min_frequency=1,
    max_frequency=25,
    dj=0.1,
)

In [ ]:
p1(data)
p2(data)
p3(data)
p4(data);

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
tfa.plotting.spectrum(data, levels=np.linspace(-6, -3, 20), ax=ax)

## EFIx_TCT

In [ ]:
data_params = dict(
    collection="SW_EXPT_EFIA_TCT02",
    measurements=["Ehx", "Ehy", "Ehz", "Quality_flags"],
    start_time=dt.datetime(2015, 3, 14, 12, 5, 0),
    end_time=dt.datetime(2015, 3, 14, 12, 30, 0),
    server_url="https://vires.services/ows",
    options=dict(asynchronous=False, show_progress=False),
)
data = create_paldata(PalDataItem.from_vires(**data_params))

In [ ]:
p1 = tfa.processes.Preprocess()
p1.set_config(
    dataset="SW_EXPT_EFIA_TCT02",
    active_variable="Eh_XYZ",
    active_component=2,
    sampling_rate=2,
)
p2 = tfa.processes.Clean()
p2.set_config(
    window_size=300,
    method="iqr",
    multiplier=1,
)
p3 = tfa.processes.Filter()
p3.set_config(
    cutoff_frequency=20 / 1000,
)
p4 = tfa.processes.Wavelet()
p4.set_config(
    min_frequency=20 / 1000,
    max_frequency=200 / 1000,
    dj=0.1,
)
p1(data)
p2(data)
p3(data)
p4(data);

In [ ]:
tfa.plotting.time_series(data)

In [ ]:
tfa.plotting.spectrum(data, levels=np.linspace(-2, 2, 20))

## AUX_OBS

In [ ]:
data_params = dict(
    collection="SW_OPER_AUX_OBSM2_:HRN",
    measurements=["B_NEC"],
    models=["Model='CHAOS-Core'+'CHAOS-Static'"],
    start_time=dt.datetime(2015, 3, 14, 0, 0, 0),
    end_time=dt.datetime(2015, 3, 14, 23, 59, 59),
    server_url="https://vires.services/ows",
    options=dict(asynchronous=False, show_progress=False),
)
data = create_paldata(PalDataItem.from_vires(**data_params))

p1 = tfa.processes.Preprocess()
p1.set_config(
    dataset="SW_OPER_AUX_OBSM2_:HRN",
    active_variable="B_NEC_res_Model",
    active_component=0,
    sampling_rate=1 / 60,
    remove_model=True,
)
p2 = tfa.processes.Clean()
p2.set_config(
    window_size=10,
    method="iqr",
    multiplier=1,
)
p3 = tfa.processes.Filter()
p3.set_config(
    cutoff_frequency=0.001,
)
p4 = tfa.processes.Wavelet()
p4.set_config(
    min_scale=1000 / 8,
    max_scale=1000 / 1,
    dj=0.1,
)

p1(data)
p2(data)
p3(data)
p4(data)

tfa.plotting.spectrum(data, levels=np.linspace(-2, 2))